#### Requirements 

In [14]:
# !pip install tensorflow 
# !pip install numpy 
# !pip install pandas
# !pip install matplotlib
# !pip install scikit-learn
# !pip install seaborn


#### GPU usage for tf

In [15]:
!nvidia-smi # veriying if NVIDEA drive and CUDA runtime loads 

Wed Jul  2 20:52:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060        Off |   00000000:01:00.0  On |                  N/A |
| 32%   36C    P3             N/A /  115W |     454MiB /   8188MiB |      8%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

# 1) Ver todas as GPUs
gpus = tf.config.list_physical_devices("GPU")
print("GPUs detectadas:", gpus)

if gpus:
    # 2) (Opcional) limitar a visão só à primeira GPU
    tf.config.set_visible_devices(gpus[0], "GPU")

    # 3) (Recomendado) liberar memória sob demanda
    tf.config.experimental.set_memory_growth(gpus[0], True)

GPUs detectadas: []


#### Other libs

In [17]:
# LIBS

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import TimeSeriesSplit
import keras

register_matplotlib_converters()
sns.set_style("darkgrid")

plt.rc("figure", figsize=(16, 6))
plt.rc("font", size=13)

from matplotlib.pyplot import figure

figure(figsize = (16, 6), dpi = 100)

<Figure size 1600x600 with 0 Axes>

<Figure size 1600x600 with 0 Axes>

#### Model improvement

In [18]:
#gridsearch
def grid_search_cv(modelo, units, X_train, learning_rates, y_train, epochs_list, batch_sizes, patiences, model_name):
    best_loss = float('inf')
    best_params = {}
    for lr in learning_rates:
        for epochs in epochs_list:
            for batch_size in batch_sizes:
                for patience in patiences:
                    model = modelo(units, X_train, lr)
                    histories = fit_model_with_cross_validation(model, X_train, y_train, model_name, patience, epochs, batch_size)
                    mean_history = calculate_mean_history(histories)
                    val_loss = min(mean_history['val_loss'])
                    print("Val Loss: ", val_loss, "learning rate: ", lr, "epochs: ",  epochs, "batch_size: " , batch_size, "patience: ", patience)
                    if val_loss < best_loss:
                        best_loss = val_loss
                        best_params = {'learning_rate': lr, 'epochs': epochs, 'batch_size': batch_size, 'patience': patience} 
    print('O modelo '+model_name+ ' tem como melhores parametros os seguintes: learning_rate '+ str(best_params['learning_rate'])+' epochs: '+ str(best_params['epochs'])+' batch_size: '+ str(best_params['batch_size'])+ ' patience: '+ str(best_params['patience']))
    return best_params

#validação cruzada
def fit_model_with_cross_validation(model, xtrain, ytrain, model_name, patience, epochs, batch_size):
    tscv = TimeSeriesSplit(n_splits=5)
    fold = 1
    histories = []
    for train_index, val_index in tscv.split(xtrain):
        x_train_fold, x_val_fold = xtrain[train_index], xtrain[val_index]
        y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, min_delta=1e-5)
        history = model.fit(x_train_fold, y_train_fold, epochs=epochs, validation_data=(x_val_fold, y_val_fold), batch_size=batch_size, callbacks=[early_stop], verbose=1)
        print('\n\nTREINAMENTO - Fold', fold, 'do modelo:', model_name)
        histories.append(history)
        fold += 1   
    return histories 

# calcula a media das metricas obtidas nos historys - validação cruzada
def calculate_mean_history(histories):
    mean_history = {'loss': [], 'root_mean_squared_error': [], 'val_loss': [], 'val_root_mean_squared_error': []}
    for fold_history in histories:
        for key in mean_history.keys():
            mean_history[key].append(fold_history.history[key])
    for key, values in mean_history.items():
        max_len = max(len(val) for val in values)
        for i in range(len(values)):
            if len(values[i]) < max_len: #caso em que nao se treina todas as epocas (patience)
                values[i] += [values[i][-1]] * (max_len - len(values[i])) #completa o restante da lista com o ultimo valor obtido
    for key, values in mean_history.items():
        mean_history[key] = [sum(vals) / len(vals) for vals in zip(*values)]
    
    return mean_history


#### LSTM construction

In [19]:
# Create input dataset
# The input shape should be [samples, time steps, features
def create_dataset (X, look_back = 3):
    Xs, ys = [], []
    
    for i in range(len(X)-look_back):
        v = X[i:i+look_back]
        Xs.append(v)
        ys.append(X[i+look_back])
        
    return np.array(Xs), np.array(ys)

# Create LSTM model
def create_lstm(units, train, learning_rate): 
    model = Sequential() 
    # Old Config
    model.add(LSTM(units = units, return_sequences = True, input_shape = [train.shape[1], train.shape[2]]))
    model.add(LSTM(units = units)) 
    # model.add(Dropout(0.2))
    model.add(Dense(1))
    # model.compile(loss=MeanSquaredError(), optimizer = Adam(learning_rate=learning_rate), metrics=[RootMeanSquaredError()])
    model.compile(
        loss=Huber(delta=0.25),  # delta define quando a perda muda de quadrática para linear
        optimizer=Adam(learning_rate=learning_rate),
        metrics=[RootMeanSquaredError()]
    )
    
    return model

#treinamento do modelo
def fit_model(model, xtrain, ytrain, model_name, patience, epochs, batch_size ):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = patience, restore_best_weights=True)
    history = model.fit(xtrain, ytrain, epochs = epochs, validation_split = 0.2, batch_size = batch_size, shuffle = True, callbacks=[early_stop]) 
    print('\n\nTREINAMENTO: ' + model_name)
    return history

# Make prediction
def prediction(model, xtest, ytest, myscaler, model_name, link): 
    prediction = model.predict(xtest) 
    prediction = myscaler.inverse_transform(prediction) 
    # dataframe_prediction = pd.DataFrame(data={'Predições':prediction.flatten()})
    dataframe_prediction = pd.DataFrame(data={'Prediction':prediction.flatten(), 'Test':ytest.flatten()})
    #save_path = os.path.join('..', '..', 'predicoes', f'prediction {model_name} {link}.csv') 
    save_path = os.path.join('..', '..', 'results', 'bi-lstm', 'forecast', f'prediction {model_name} {link}.csv') 
    dataframe_prediction.to_csv(save_path)
    return prediction


# Calculate MAE and RMSE
def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    nrmse = rmse/ np.max(actual)
    mae = np.abs(errors).mean()
    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('Normalized Root Mean Square Error: {:.4f}%'.format(nrmse*100))
    print('')

    return rmse, mae, nrmse, model_name

# ===================================================================================
# NOVA FUNÇÃO: valida métrica apenas em pontos com dado real (mask == True)
# ===================================================================================
def validate_missing_data_prediction(predictions, actual, mask, model_name):
    """
    Calcula RMSE, MAE e NRMSE APENAS nos pontos onde há dado real (mask==True).

    Parameters
    ----------
    predictions : np.ndarray  (shape: [N, 1] ou [N])
        Vetor de predições PURO, sem mistura com pontos reais.
    actual      : np.ndarray  (shape: [N, 1] ou [N])
        Vetor de valores reais correspondentes.
    mask        : np.ndarray  (shape: [N], dtype=bool)
        True  -> ponto com dado real
        False -> ponto ausente (gap) onde não devemos avaliar.
    model_name  : str
        Identificador do modelo (ex.: 'LSTM')

    Returns
    -------
    (rmse, mae, nrmse, model_name)
    """
    # Garante formato 1-D
    predictions = predictions.flatten()
    actual      = actual.flatten()
    mask        = mask.astype(bool).flatten()

    # Seleciona apenas os pontos válidos
    preds_valid = predictions[mask]
    acts_valid  = actual[mask]

    if len(acts_valid) == 0:
        print(f"{model_name}: NÃO HÁ PONTOS REAIS PARA AVALIAÇÃO NESTA JANELA!")
        return np.nan, np.nan, np.nan, model_name

    errors = preds_valid - acts_valid
    mse    = np.square(errors).mean()
    rmse   = np.sqrt(mse)
    nrmse  = rmse / ((acts_valid.max() - acts_valid.min()) + 1e-12)
    mae    = np.abs(errors).mean()

    print(model_name + ' (missing-aware):')
    print(f'MAE:  {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'NRMSE:{nrmse*100:.4f}%\n')

    return rmse, mae, nrmse, model_name

def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    nrmse = rmse/ ((np.max(actual))-(np.min(actual)))
    mae = np.abs(errors).mean()
    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('Normalized Root Mean Square Error: {:.4f}%'.format(nrmse*100))
    print('')

    return rmse, mae, nrmse, model_name


#### Utils

In [20]:
def bits_para_megabits(df, col_vaz):
    # Dados em MegaBits/s e fill
    df[col_vaz] = df[col_vaz]/1000000
    df[col_vaz] = df[col_vaz].replace(-1, df[col_vaz].mean())
    df[col_vaz] = df[col_vaz].fillna(df[col_vaz].mean())

    return df

def linear_interpolation(df, limit_direction='both', method='linear'):
    df_imputed = df.interpolate(method=method, limit_direction=limit_direction)

    df['Throughput'] = df['Throughput'].fillna(df_imputed['Throughput'])

    return df


#### Plots and visualizations

In [21]:
                
def visualizacao_series(df, col_vazao, titulo):
    df[col_vazao].plot(figsize=(18,6))
    plt.title(titulo)
    plt.ylabel('Vazao (Mbits/s)')
    plt.legend() 
    plt.show()

#plotar os graficos da media dos treinamentos por epocas: validação cruzada
def plot_loss_cv(mean_history, model_name, link):
    epochs = range(1, len(mean_history['loss']) + 1)
    plt.plot(epochs, mean_history['loss'], label='Train Loss')
    plt.plot(epochs, mean_history['val_loss'], label='Validation Loss')
    plt.title('Mean Training and Validation Loss for '+' '+link + ' '+ model_name)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_rmse_cv(mean_history):
    epochs = range(1, len(mean_history['root_mean_squared_error']) + 1)
    plt.plot(epochs, mean_history['root_mean_squared_error'], label='Train RMSE')
    plt.plot(epochs, mean_history['val_root_mean_squared_error'], label='Validation RMSE')
    plt.title('Mean Training and Validation RMSE')
    plt.xlabel('Epoch')
    plt.ylabel('RMSE')
    plt.legend()
    plt.show()

########################################### plote dos graficos de treinamento ###################################################################################
 #Plot train loss and validation loss
def plot_loss(history, model_name, link):
     plt.figure(figsize = (15, 6), dpi=100)
     plt.plot(history.history['loss'])
     plt.plot(history.history['val_loss'])
     plt.title('Model Train vs Validation Loss for '+' '+link + ' '+ model_name)
     plt.ylabel('Loss')
     plt.xlabel('Epoch')
     plt.legend(['Train loss', 'Validation loss'], loc='upper right')
def plot_rmse(history, model_name, link):
     plt.figure(figsize = (15, 6), dpi=100)
     plt.plot(history.history['rmse'])
     plt.plot(history.history['val_rmse'])
     plt.title('Model Train vs RMSE for '+' '+link + ' '+ model_name)
     plt.ylabel('rmse')
     plt.xlabel('Epoch')
     plt.legend(['Train rmse', 'Validation loss'], loc='upper right')
################################################################################################################################################################
 

def plot_future(predictionLSTM, y_test, link):
    plt.figure(figsize=(15, 6), dpi=100)
    range_future = len(y_test)
    plt.plot(np.arange(range_future), np.array(y_test), label='Test data')
    plt.plot(np.arange(range_future), np.array(predictionLSTM), label='LSTM')
    # dict_to_dataframe_prediction = {
    #     # "range_future": np.arange(range_future),
    #     f"prediction{model_name}": np.array(prediction.squeeze())
    # }
    
    plt.title('Test data vs prediction for '+ link)
    plt.legend(loc='upper left')
    plt.xlabel('Time')
    plt.ylabel('Mbis/s')
    save_path = os.path.join('..', '..', 'results', 'bi-lstm', 'plots', link + '.png')
    save_path = os.path.normpath(save_path)  

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    try:
        plt.savefig(save_path)
        print(f"A figura foi salva com sucesso em: {save_path}")
    except Exception as e:
        print(f"Erro ao salvar a figura: {e}")
    plt.show()

    # #Tenta salvar a fig
    # save_path = os.path.join('..', '..', 'graficos', 'predicoes', 'round_2', 'graficos', link + '.png')

    # #save_path = '../../graficos/predicoes/round_2/graficos/' + link + '.png'
    # try:
    #     plt.savefig(save_path)
    #     print(f"A figura foi salva com sucesso em: {save_path}")
    # except Exception as e:
    #     print(f"Erro ao salvar a figura: {e}")

    # plt.show()


# Plot test data vs prediction
# def plot_future(predictionGRU, predictionLSTM, y_test, link):
#     plt.figure(figsize=(15, 6), dpi=100)
#     range_future = len(y_test)
#     plt.plot(np.arange(range_future), np.array(y_test), label='Test data')
#     plt.plot(np.arange(range_future), np.array(predictionGRU), label='GRU')
#     plt.plot(np.arange(range_future), np.array(predictionLSTM), label='LSTM')
#     # dict_to_dataframe_prediction = {
#     #     # "range_future": np.arange(range_future),
#     #     f"prediction{model_name}": np.array(prediction.squeeze())
#     # }
    
#     plt.title('Test data vs prediction for '+ link)
#     plt.legend(loc='upper left')
#     plt.xlabel('Time')
#     plt.ylabel('Mbis/s')

#     #Tenta salvar a fig
#     save_path = '../../graficos/predicoes/round_2/graficos/' + link + '.png'
#     try:
#         plt.savefig(save_path)
#         print(f"A figura foi salva com sucesso em: {save_path}")
#     except Exception as e:
#         print(f"Erro ao salvar a figura: {e}")

#     plt.show()
    

#### Data manipulation

##### Paths

In [22]:
# Paths
TRAINING_OUTPUT = os.path.join('training_output.txt')
THROUGHPUT_DATASETS = os.path.join('..', '..', 'datasets', 'test-recursive-lstm-test')
MODEL = os.path.join("..", "..", 'modelo_salvo')
METRICS = os.path.join('..', '..', 'results', 'bi-lstm', 'evaluation_rmse_mae_2.json')

In [23]:
#função para salvar o modelo
def save_model(model, directory, substring_desejada, modelo):
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{substring_desejada +modelo} - final_model.keras')
    model.save(file_path)
    print(f"Modelo salvo como '{file_path}'")

### Main
#### Model training and predcition 

#### Funtional with part of new solution

In [24]:
def walk_forward_validation_hybrid(model, scaler, train_data, test_data, mask_test, look_back, window_size):
    """
    Executa walk-forward validation com imputação híbrida preservando estado temporal
    
    Parâmetros:
    model -- Modelo LSTM pré-treinado
    scaler -- Scaler usado na normalização
    train_data -- Dados de treino normalizados
    test_data -- Dados de teste normalizados
    mask_test -- Máscara de dados faltantes no teste
    look_back -- Tamanho da janela histórica
    window_size -- Tamanho da janela de predição
    
    Retorna:
    predictions -- Lista de previsões em escala original
    """
    # Estado inicial: últimos pontos do treino
    state = train_data[-look_back:].reshape(1, look_back, 1)
    predictions = []
    
    for i in range(0, len(test_data), window_size):
        # Seleciona janela atual
        end_idx = min(i + window_size, len(test_data))
        window_data = test_data[i:end_idx]
        window_mask = mask_test[i:end_idx]
        
        window_preds = []
        current_state = state
        
        # Predição passo-a-passo dentro da janela
        for j in range(len(window_data)):
            # Faz predição com estado atual
            pred_scaled = model.predict(current_state, verbose=0)
            pred_orig = scaler.inverse_transform(pred_scaled)[0,0]
            window_preds.append(pred_orig)
            
            # Atualiza estado com dado real ou predição
            if window_mask[j]:
                new_point = window_data[j]
            else:
                new_point = pred_scaled[0,0]
            
            # Atualiza estado: remove ponto mais antigo, adiciona novo
            current_state = np.roll(current_state, -1, axis=1)
            current_state[0, -1, 0] = new_point
        
        predictions.extend(window_preds)
        state = current_state  # Mantém estado para próxima janela
    
    return predictions


In [25]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping

# Certifique-se de importar funções auxiliares usadas no script
# from utils import bits_para_megabits, linear_interpolation, create_dataset, grid_search_cv
# from utils import create_lstm, walk_forward_validation_hybrid, validate_missing_data_prediction
# from utils import save_model, calculate_mean_history

# Variáveis globais/configuradas
LOOK_BACK = 3
WINDOW_SIZE = 28  # Tamanho da janela de predição
# TRAINING_OUTPUT = 'training_output.txt'  # Ajuste conforme necessário
# THROUGHPUT_DATASETS = './datasets'  # Caminho correto
# MODEL = './saved_models'  # Caminho correto para salvar os modelos

# Funções de plot

def plot_loss(history, title):
    plt.figure(figsize=(10, 5))
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


def plot_predictions(y_true, y_pred, title):
    plt.figure(figsize=(12, 6))
    plt.plot(y_true, label='True Values')
    plt.plot(y_pred, label='Predictions')
    plt.title(title)
    plt.xlabel('Time Steps')
    plt.ylabel('Throughput (Mbps)')
    plt.legend()
    plt.show()

# Redirecionar saída padrão
orig_stdout = sys.stdout

with open(TRAINING_OUTPUT, 'w', encoding='utf-8') as f:
    sys.stdout = f

    diretorio_raiz = THROUGHPUT_DATASETS
    evaluation = {}

    for pasta_raiz, subpastas, arquivos in os.walk(diretorio_raiz):
        for arquivo in arquivos:
            if arquivo.endswith('.csv'):
                caminho_arquivo = os.path.join(pasta_raiz, arquivo)
                try:
                    partes = caminho_arquivo.split(os.sep)
                    if len(partes) >= 6:
                        substring_desejada = partes[4] + ' - ' + partes[5]
                    else:
                        substring_desejada = arquivo.replace('.csv', '')

                    # Carga dos dados
                    df = pd.read_csv(caminho_arquivo, index_col='Timestamp')

                    # Remover colunas desnecessárias, se houver
                    if '0' in df.columns:
                        df.drop('0', axis=1, inplace=True)

                    # Pré-processamento
                    bits_para_megabits(df, 'Throughput')

                    # Criar máscara ANTES de qualquer manipulação
                    mask_total = ~(df['Throughput'].isna() | (df['Throughput'] == -1))

                    # Split treino-teste
                    train_size = int(len(df.index) * 0.8)
                    train_data = df.iloc[:train_size].copy()
                    test_data = df.iloc[train_size:].copy()

                    # Interpolação APENAS no treino
                    train_data = linear_interpolation(train_data)

                    # Normalização
                    scaler = MinMaxScaler().fit(train_data[['Throughput']])
                    train_scaled = scaler.transform(train_data[['Throughput']])
                    test_scaled = scaler.transform(test_data[['Throughput']])

                    mask_test = mask_total.iloc[train_size:].values

                    # Treinamento do modelo
                    X_train, y_train = create_dataset(train_scaled, LOOK_BACK)

                    best_params = grid_search_cv(
                        create_lstm, 64, X_train, [1e-3],
                        y_train, [100,300,500], [32, 64, 128], [10], 'LSTM'
                    )

                    # Treinar modelo final
                    model = create_lstm(64, X_train, best_params['learning_rate'])
                    history = model.fit(
                        X_train, y_train,
                        epochs=best_params['epochs'],
                        batch_size=best_params['batch_size'],
                        validation_split=0.2,
                        callbacks=[EarlyStopping(
                            monitor='val_loss',
                            patience=best_params['patience'],
                            restore_best_weights=True
                        )]
                    )

                    # Predição walk-forward
                    predictions = walk_forward_validation_hybrid(
                        model, scaler, train_scaled, test_scaled,
                        mask_test, LOOK_BACK, WINDOW_SIZE
                    )

                    # Avaliação
                    min_len = min(len(predictions), len(test_scaled) - LOOK_BACK)
                    y_test_valid = test_scaled[LOOK_BACK:LOOK_BACK + min_len]
                    mask_eval = mask_test[LOOK_BACK:LOOK_BACK + min_len]

                    y_test_orig = scaler.inverse_transform(y_test_valid)
                    predictions_arr = np.array(predictions[:min_len]).reshape(-1, 1)

                    lstm_eval = validate_missing_data_prediction(
                        predictions_arr, y_test_orig, mask_eval, 'LSTM'
                    )

                    evaluation[f"{substring_desejada}, {lstm_eval[3]}"] = lstm_eval[:3]

                    print(f"RMSE para {substring_desejada}: {lstm_eval[0]:.4f}")
                    print(f"MAE para {substring_desejada}: {lstm_eval[1]:.4f}")
                    print(f"NRMSE para {substring_desejada}: {lstm_eval[3]:.4f}")

                    # Salvar e plotar
                    save_model(model, MODEL, substring_desejada, 'LSTM')
                    plot_loss(history.history, f'LSTM Loss - {substring_desejada}')
                    plot_predictions(y_test_orig, predictions_arr, f'LSTM Predictions - {substring_desejada}')

                except Exception as e:
                    print(f"Erro no arquivo {arquivo}: {str(e)}")

    # Restaurar stdout
    sys.stdout = orig_stdout


/home/clara/Documentos/recursive-prediction/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/tmp/ipykernel_905141/2259065887.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_state[0, -1, 0] = new_point
/tmp/ipykernel_905141/2259065887.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_state[0, -1, 0] = new_point
/tmp/ipykernel_905141/2259065887.py:45: DeprecationWarning: Conversion of an 